#Book Sunmaries using Ollama

In [ ]:
from bs4 import BeautifulSoup
import requests
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
MODEL = "llama3.2"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

client = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [ ]:
def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:10_000]


In [ ]:
system_prompt = """
You are a leader in your local book club handling the usual
tasks of book club group leader i.e scheduling, meet-ups, researching
and coming up with books that will be read next etc
"""

user_prompt = """
Here are the contents of a books website.
Provide a short summary of a few books on the website.
Capture the book title, author and a short synopsis of each book.
"""

In [ ]:
def summarize(url):
    content = fetch_website_contents(url)
    response = client.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt + f"\n\nWebsite content: {content}"},
        ]
    )
    return response.choices[0].message.content

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://thestorygraph.com")